In [12]:
import torch
torch.cuda.empty_cache()
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [13]:

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=getattr(torch, "float16"),
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Let's use FAISS for the RAG

In [14]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = [
    "https://uk.investing.com/news/stock-market-news/iran-seizes-oil-tanker-involved-in-usiran-dispute-in-gulf-of-oman-3297614",
    "https://uk.investing.com/news/world-news/biden-warns-of-more-strikes-on-yemens-houthis-if-red-sea-attacks-persist-3297572",
    "https://uk.investing.com/news/commodities-news/oil-prices-slips-lower-consolidating-ahead-of-dataheavy-week-3298043",
    "https://uk.investing.com/news/commodities-news/oil-slips-investors-eye-mideast-developments-3298049",
    "https://uk.investing.com/news/stock-market-news/qatar-pauses-red-sea-tankers-after-western-strikes-on-houthis-3298339",
    "https://uk.investing.com/news/stock-market-news/factboxshipping-firms-react-to-houthi-attacks-in-the-red-sea-3298396"
]
# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
# Converts HTML to plain text 
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [30]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredURLLoader(urls=articles)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(splits)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# Create a Prompt and LLMChain

In [ ]:
prompt_template = """
### [INST] Instruction: Answer the question based on your oil knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

Now with context:

In [31]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("Should I invest in oil?")

/home/balalo/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [32]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence

In [33]:
result['context']

[Document(page_content='Markets were now awaiting any potential retaliation by the Houthis for last week’s strikes, after the group said it will continue targeting ships headed towards Israel.\n\n"While geopolitical risks are certainly building, we are still not seeing a reduction in oil supply as a result of developments in the region," said analysts at ING, in a note. "But, the more escalation we see in the region, the more the market will have to start pricing\xa0in a larger risk of supply disruptions."\n\nOil prices were also nursing a weak start to 2024 after tumbling over 10% in the past year, as markets remained convinced that global crude demand will see little improvement this year amid pressure from high interest rates, cooling economic growth and sticky inflation.\n\nChina, US data awaited for more demand cues\n\nFocus was now squarely on key upcoming economic readings from the U.S. and China this week, for more cues on the potential path of demand.', metadata={'source': 'ht

In [34]:
print(result['text'])


Based on the provided information, it appears that there are several factors affecting the oil market, including geopolitical risks, economic conditions, and weather-related disruptions. While oil prices have been volatile in recent years, it is important to consider the long-term trends and potential risks before making any investment decisions.

It is worth noting that investing in oil can be a complex and potentially high-risk endeavor, as the market is subject to a wide range of factors that can affect supply and demand. Additionally, the environmental and social impacts of oil production and consumption should also be considered when evaluating the potential benefits and drawbacks of investing in this commodity.

Ultimately, whether or not you should invest in oil depends on your individual financial goals, risk tolerance, and understanding of the market dynamics. It may be beneficial to consult with a financial advisor or conduct further research before making any investment dec